In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#este es el set de entrtenammiento completo el cual se prepocesara y se generar 2 nuevos CSV  X e y para train
df_train = pd.read_csv('data/raw/train_dataset.csv')

#estos son los set de datos de test los cuales se preprocesaran y se generar 2 nuevos CSV para test
df_test = pd.read_csv('data/raw/x_test.csv')
tmp_y_test_final = pd.read_csv('data/raw/y_test.csv')

Primero se imputan valores en las columnas que contienen nulos en el dataset DE PRUEBA

year_built, energy_star_rating, direction_max_wind_speed, direction_peak_wind_speed, max_wind_speed, days_with_fog


In [ ]:
# Se obtienen los valores antes de imputar para poder imputra la misma media en ambos set de datos
a = df_train['year_built'].mean()
b = df_train['energy_star_rating'].mean()
c = df_train['direction_max_wind_speed'].mean()
d = df_train['direction_peak_wind_speed'].mean()
e = df_train['max_wind_speed'].mean()
f = df_train['days_with_fog'].mean()

In [ ]:
df_train['year_built'].fillna(a, inplace=True)
df_train['energy_star_rating'].fillna(b, inplace=True)
df_train['direction_max_wind_speed'].fillna(c, inplace=True)
df_train['direction_peak_wind_speed'].fillna(d, inplace=True)
df_train['max_wind_speed'].fillna(e, inplace=True)
df_train['days_with_fog'].fillna(f, inplace=True)

Ahoar se imputan los valores nulos en el DF de TEST pero con los valores del DF de TRAIN

In [ ]:
a = df_test['year_built'].mean()
b = df_test['energy_star_rating'].mean()
c = df_test['direction_max_wind_speed'].mean()
d = df_test['direction_peak_wind_speed'].mean()
e = df_test['max_wind_speed'].mean()
f = df_test['days_with_fog'].mean()

In [ ]:
df_test['year_built'].fillna(a, inplace=True)
df_test['energy_star_rating'].fillna(b, inplace=True)
df_test['direction_max_wind_speed'].fillna(c, inplace=True)
df_test['direction_peak_wind_speed'].fillna(d, inplace=True)
df_test['max_wind_speed'].fillna(e, inplace=True)
df_test['days_with_fog'].fillna(f, inplace=True)

In [ ]:
#se genera el nuevo DF sin las columnas categoricas y la variable objetivo
temp_X_train = df_train.drop(columns=['State_Factor','building_class', 'facility_type', 'site_eui', 'id'])

In [ ]:
# se genera el nuevo DF de TEST sin las variables categoricas
temp_X_test = df_test.drop(columns=['State_Factor', 'building_class', 'facility_type','id'])

In [ ]:
# se genera el nuevo DF de train SOLO CON LA VARIABLE OBJETIVO
temp_y_train = df_train.iloc[:, -1]

In [ ]:
temp_y_test = tmp_y_test_final.iloc[:, -1]

In [ ]:
temp_X_train.shape

In [ ]:
temp_X_test.shape

In [ ]:
temp_y_train.shape

In [ ]:
temp_y_test.shape

Se calcula el Z-score para tratar de eliminar los outliers

In [ ]:
from scipy import stats
import numpy as np

In [ ]:
z = np.abs(stats.zscore(temp_X_train))
print(z)

In [ ]:
temp_X_train = temp_X_train[(z < 3).all(axis=1)]

In [ ]:
temp_X_train.shape

In [ ]:
temp_X_test = temp_X_test[(z < 3).all(axis=1)]

In [ ]:
temp_X_test.shape

In [ ]:
temp_y_train = temp_y_train[(z < 3).all(axis=1)]

In [ ]:
temp_y_train.shape

In [ ]:
temp_y_test = temp_y_test[(z < 3).all(axis=1)]

In [ ]:
temp_y_test.shape

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [ ]:
# compute the vif for all given features
def compute_vif(df, considered_features):
    
    X = df[considered_features]
    # the calculation of variance inflation requires a constant
    X['intercept'] = 1
    
    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['Variable']!='intercept']
    return vif

In [ ]:
# featires altamente correlacionadas y que deberia ser borradas
considered_features = ['january_avg_temp', 
'february_min_temp', 
'february_avg_temp', 
'march_min_temp',
'heating_degree_days', 
'cooling_degree_days',
'july_avg_temp', 
'august_avg_temp', 
'days_below_20F',
'days_below_30F', 
'days_below_10F', 
'direction_peak_wind_speed', 
'direction_max_wind_speed',
'max_wind_speed'
]

# compute vif 
compute_vif(temp_X_train, considered_features).sort_values('VIF', ascending=False)

In [ ]:
X_train_final = temp_X_train.drop(columns=['january_avg_temp','february_min_temp', 
                                          'february_avg_temp', 'march_min_temp','heating_degree_days',
                                          'cooling_degree_days','july_avg_temp', 'august_avg_temp', 
                                          'days_below_20F','days_below_30F', 'days_below_10F', 'direction_peak_wind_speed',
                                          'direction_max_wind_speed','max_wind_speed'])

In [ ]:
X_test_final = temp_X_test.drop(columns=['january_avg_temp','february_min_temp', 
                                          'february_avg_temp', 'march_min_temp','heating_degree_days',
                                          'cooling_degree_days','july_avg_temp', 'august_avg_temp', 
                                          'days_below_20F','days_below_30F', 'days_below_10F', 'direction_peak_wind_speed',
                                          'direction_max_wind_speed','max_wind_speed'])

Probamos haciendo una standarizacion normal comun y corriente ya que en teoria se eliminaron los outlier

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
transformer = StandardScaler().fit(X_train_final)
temp_X_train_scaled = transformer.transform(X_train_final)

In [ ]:
transformer2 = StandardScaler().fit(X_test_final)
temp_X_test_scaled = transformer2.transform(X_test_final)

In [ ]:
#se debe transforma a un dataframe ya que el metodo transorm lo retorna en un ndarray
temp_X_train_scaled_df = pd.DataFrame(temp_X_train_scaled, columns=X_train_final.columns)
temp_X_train_scaled_df.describe()

In [ ]:
#se debe transforma a un dataframe ya que el metodo transorm lo retorna en un ndarray
temp_X_test_scaled_df = pd.DataFrame(temp_X_test_scaled, columns=X_test_final.columns)
temp_X_test_scaled_df.describe()

In [ ]:
temp_X_train_scaled_df.shape

In [ ]:
temp_X_test_scaled_df.shape

In [ ]:
temp_y_train.shape

In [ ]:
temp_y_test.shape

In [ ]:
#X_train_final.to_csv('data/preprocesada/X_train.csv')
#X_test_final.to_csv('data/preprocesada/X_test.csv')

temp_X_train_scaled_df.to_csv('data/preprocesada/X_train.csv')
temp_X_test_scaled_df.to_csv('data/preprocesada/X_test.csv')

temp_y_train.to_csv('data/preprocesada/y_train.csv')
temp_y_test.to_csv('data/preprocesada/y_test.csv')